In [1]:
#- Stuff Documents 체인을 사용하여 완전한 RAG 파이프라인을 구현하세요. 체인을 수동으로 구현해야 합니다.
#- 체인에 ConversationBufferMemory를 부여합니다.
#- 이 문서를 사용하여 RAG를 수행하세요: https://gist.github.com/serranoarevalo/5acf755c2b8d83f1707ef266b82ea223
#- 체인에 다음 질문을 합니다:
#	Aaronson 은 유죄인가요?
#	그가 테이블에 어떤 메시지를 썼나요?
#	Julia 는 누구인가요?

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI( temperature=0.3, streaming=True )

memory = ConversationBufferMemory(memory_key="history")

cache_dir = LocalFileStore("./.cache/")

loader = UnstructuredFileLoader("./files/assignment4.txt")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=500,
    chunk_overlap=100,
)

splitted_docs = loader.load_and_split(text_splitter=splitter) # split

embeddings = OpenAIEmbeddings() # 오픈ai 임베딩 사용

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings,cache_dir) # 임베딩을 디렉토리에 캐싱

vectorstore = FAISS.from_documents(splitted_docs, cached_embeddings)

retriever = vectorstore.as_retriever()



[nltk_data] Downloading package punkt to /home/hyewona/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/hyewona/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
/tmp/ipykernel_95658/3421180497.py:27: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings() # 오픈ai 임베딩 사용


In [ ]:
def load_memory(_):
    return memory.load_memory_variable({})["history"]

prompt = ChatPromptTemplate.from_messages(
    [("system", "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{document}"),
    MessagesPlaceholder(variable_name="history"),
    ("human","{question}")]
)

chain = (
    {
        "document": retriever,
        "question": RunnablePassthrough(),
        "history": RunnablePassthrough.assign(history=load_memory)
    }
) | prompt | llm

def invoke_chain(question):
    #print(type(question))
    chain.invoke({"question":question})
    memory.save_content(
        {"input":question}, {"output":result.content}
    )
    print(result)

invoke_chain("Is Aaronson guilty?")
invoke_chain("What message did he write in the table?")
invoke_chain("Who is Julia?")

TypeError: expected string or buffer